# Microstructure Signal Visualization

This notebook visualizes the microstructure signals generated by the feature pipeline. The goal is to qualitatively inspect the signals against the price chart to see if they behave as expected.

**Signals Visualized:**
- Price (Close)
- Volume
- Volatility Regime (`ms_vol_regime`)
- Chop Score (`ms_chop`)
- Bursts (`ms_burst_up`, `ms_burst_down`)
- Exhaustion (`ms_exhaustion_up`, `ms_exhaustion_down`)
- Parabolic Trend (`ms_parabolic_trend`)

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Ensure the project root is in the Python path
ROOT = Path().resolve().parents[2]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from finantradealgo.features.feature_pipeline import (
    build_feature_pipeline_from_system_config,
    FeaturePipelineConfig
)

# Configure plots
plt.style.use('seaborn-v0_8-darkgrid')
print("Imports complete and environment set up.")

In [ ]:
# --- Configuration ---
SYMBOL = "BTCUSDT"  # or "AIAUSDT"
TIMEFRAME = "15m"

# --- Run the full feature pipeline ---
print("Running the feature pipeline...")
# We enable all relevant features for this visualization
pipeline_cfg = FeaturePipelineConfig(
    use_microstructure=True,
    use_base=True,
    use_ta=True,
    use_candles=True,
    use_osc=True,
    drop_na=True, # Drop NaNs for cleaner plots
    # Disable features we don't need for this viz
    use_htf=False,
    use_market_structure=False,
    use_external=False,
    use_rule_signals=False,
)

df, _ = build_feature_pipeline_from_system_config(
    pipeline_cfg=pipeline_cfg, 
    symbol=SYMBOL, 
    timeframe=TIMEFRAME
)

print(f"Pipeline complete. DataFrame shape: {df.shape}")
df.set_index('timestamp', inplace=True)
df.tail()

In [ ]:
# --- Visualization ---
print("Plotting signals...")

# Select a recent slice of data for better visibility
df_slice = df.last('200D') # Adjust the time window as needed

signals_to_plot = [
    'ms_chop',
    'ms_vol_regime',
    'ms_burst_up',
    'ms_burst_down',
    'ms_exhaustion_up',
    'ms_exhaustion_down',
    'ms_parabolic_trend'
]

# Create subplots
fig, axes = plt.subplots(
    nrows=len(signals_to_plot) + 2, # Price + Volume + signals
    ncols=1, 
    figsize=(18, 2 * (len(signals_to_plot) + 2)), 
    sharex=True
)

# Plot 1: Close Price
axes[0].plot(df_slice.index, df_slice['close'], label='Close Price', color='blue')
axes[0].set_title(f'{SYMBOL} {TIMEFRAME} Price')
axes[0].legend()

# Plot 2: Volume
axes[1].bar(df_slice.index, df_slice['volume'], label='Volume', color='gray', width=0.005)
axes[1].set_title('Volume')
axes[1].legend()

# Plot other signals
for i, signal in enumerate(signals_to_plot):
    ax = axes[i + 2]
    ax.plot(df_slice.index, df_slice[signal], label=signal, color='purple')
    ax.set_title(signal)
    ax.legend()

# Improve formatting
plt.tight_layout()
fig.autofmt_xdate()
plt.show()